In [ ]:
!pip install torch torchvision torchaudio numpy --user
!pip install easyocr --user
!pip install opencv-python==4.5.5.64 --user
!pip install os-sys --user
!pip install ipywidgets --user
!pip install pytesseract --user


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import easyocr
import pytesseract
import sys
import os
from pytesseract import Output
%matplotlib inline

In [ ]:
# Funkcje pomocnicze do wyświetlania obrazów
def plot_gray(image):
    """ Funkcja do wyświetlania obrazu w skali szarości """
    plt.figure(figsize=(16, 10))
    plt.imshow(image, cmap='Greys_r')
    plt.axis('off')
    plt.show()

In [ ]:
def plot_rgb(image):
    """ Funkcja do wyświetlania obrazu w kolorach RGB """
    plt.figure(figsize=(16, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [ ]:
# Funkcja przetwarzania obrazu przed OCR

def preprocess_image(image, scale_factor=1.5, blur_ksize=(5, 5)):
    """ Funkcja przetwarzania obrazu przed OCR """
    height, width = image.shape[:2]
    new_dimensions = (int(width * scale_factor), int(height * scale_factor))
    image_resized = cv2.resize(image, new_dimensions)
    image_blurred = cv2.GaussianBlur(image_resized, blur_ksize, 0)
    _, image_thresholded = cv2.threshold(image_blurred, 150, 255, cv2.THRESH_BINARY_INV)
    return image_thresholded


In [ ]:
# Wczytywanie obrazu

file_name = "P://a2.png"  # Podanie scieżki pliku
if not os.path.exists(file_name):
    raise FileNotFoundError(f"Nie znaleziono pliku: {file_name}")
image = cv2.imread(file_name, cv2.IMREAD_GRAYSCALE)
plot_gray(image)

In [ ]:
# Wykrywanie tekstu za pomocą EasyOCR

reader = easyocr.Reader(['pl'], gpu=True)
result = reader.readtext(image)
if result:
    print(f"EasyOCR Result: {result[0]}")

In [ ]:
# Wykrywanie tekstu za pomocą pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  #Ścieżka do Tesseract OCR
d = pytesseract.image_to_data(image, output_type=Output.DICT)
n_boxes = len(d['level'])
boxes = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)
for i in range(n_boxes):
    (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
    boxes = cv2.rectangle(boxes, (x, y), (x + w, y + h), (0, 255, 0), 1)
plot_rgb(boxes)


In [ ]:
# Wykonywanie OCR z pytesseract w języku polskim

eksport = pytesseract.image_to_string(image, lang='pol')
print("OCR Result:\n", eksport)


In [ ]:
output_file = "P://przetowrzone1.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(eksport)
print(f"Tekst zapisano do pliku: {output_file}")


In [ ]:
!pip install pymongo --user


In [ ]:
import pymongo
from pymongo import MongoClient

In [ ]:
def export_to_mongo(eksport, db_name='ocr_database', collection_name='ocr_results'):
    """ Funkcja eksportu wyników OCR do bazy danych MongoDB """
    try:
        # Połączenie z lokalną bazą MongoDB (domyślny host: localhost, port: 27017)
        client = MongoClient("mongodb://localhost:27017/")
        
        # Dostęp do bazy danych
        db = client[db_name]
        
        # Dostęp do kolekcji (tabeli)
        collection = db[collection_name]
        
        # Przygotowanie dokumentu do zapisania w MongoDB
        document = {
            "ocr_text": eksport,
            
        }
        
        # Wstawienie dokumentu do kolekcji
        collection.insert_one(document)
        print("OCR data has been successfully exported to MongoDB.")
    except Exception as e:
        print(f"Error exporting to MongoDB: {e}")


In [ ]:
# Funkcja do eksportu wyników OCR
def process_and_export_to_mongo(image):
# Eksport do MongoDB
    export_to_mongo(eksport)
    return eksport


In [ ]:
ocr_text = process_and_export_to_mongo(file_name)
print("OCR Text:\n", ocr_text)
